# 1. Import Libraries:

In [148]:
import mediapipe as mp
import numpy as np
import cv2
import os

In [149]:
text_background = (198, 63, 88)  # PURPLE
corner_color = (53, 53, 249)     # RED
text_color = (239, 239, 239)     # WHITE
border_color = (61, 147, 8)      # GREEN

# 5. Setup Folders for Collection:

In [187]:
DATA_PATH = os.path.join("Data")

# You, Busy
actions = np.array(["Yes", "WhatAreYouDoing", "TryBeing", "ToMeet", "ThankYou", "TakeCare", "SameAsYou", "Question", "Point", "Nothing", "IHear", "HowAreYou", "Hello", "Bye", "Good", "_BLANK"])

no_sequences = 40
sequence_length = 30

In [188]:
len(actions)
actions.shape[0]

16

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 6. Collect MP Keypoints:

In [ ]:
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)

def mediapipe_detections(frame, model):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame.flags.writeable = False
    results = model.process(frame)
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    return frame, results


def draw_landmarks(frame, results, color):
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)


def extract_keypoints(results):
    hand_landmarks = np.zeros(63)
    
    if results.multi_hand_landmarks:
        hand_landmarks = np.array(
            [
                [landmark.x, landmark.y, landmark.z]
                for landmark in results.multi_hand_landmarks[0].landmark
            ]
        ).flatten()

    return hand_landmarks

In [ ]:
stream_url = "http://192.168.169.196:81/stream"
cap = cv2.VideoCapture(0)

no_frames_counter = 0
no_sequences_counter = 0
index = 15
current_action = actions[index]

while True:
    _, image = cap.read()
    
    image, results = mediapipe_detections(image, hands)
    draw_landmarks(image, results, corner_color)
    right_hand = extract_keypoints(results)
    
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("s"):
        npy_path = os.path.join(DATA_PATH, current_action, str(no_sequences_counter), f"{no_frames_counter}.npy")
        np.save(npy_path, right_hand)
        no_frames_counter += 1
        if no_frames_counter == sequence_length:
            no_frames_counter = 0
            no_sequences_counter += 1
            if no_sequences_counter == no_sequences:
                break
    
    cv2.putText(image, f"Collecting Frames for '{current_action}'", (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(image, f"Video Num: {no_sequences_counter}", (15, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(image, f"Frame Num: {no_frames_counter}", (15, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow("Image", image)
    
    if key == ord("n"):
        cv2.imwrite(f"{current_action}-image-{no_frames_counter}-{no_sequences_counter}.jpg", image)
    
    if key == ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()


In [ ]:
cap.release()
cv2.destroyAllWindows()

# 7. Preprocess Data and Create Labels and Features:

In [152]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [153]:
label_map = {label:num for num, label in enumerate(actions)}

In [154]:
label_map

{'Yes': 0,
 'WhatAreYouDoing': 1,
 'TryBeing': 2,
 'ToMeet': 3,
 'ThankYou': 4,
 'TakeCare': 5,
 'SameAsYou': 6,
 'Question': 7,
 'Point': 8,
 'Nothing': 9,
 'IHear': 10,
 'HowAreYou': 11,
 'Hello': 12,
 'Bye': 13,
 'Good': 14,
 '_BLANK': 15}

In [155]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [157]:
X = np.array(sequences)
print(X.shape) # (640, 30, 63)  ==> 63  for the inner array of hand_landmarks
               #                ==> 30  for the number of sequences
               #                ==> 640 for 16 signs x 40 (folder par sign)

(640, 30, 63)


In [158]:
y = to_categorical(labels).astype(int)

In [159]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# 8. Build and Train LSTM Neural Network:

In [162]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [163]:
log_dir = os.path.join("logs")
tb_callback = TensorBoard(log_dir=log_dir) # for moritoring the NN training ==> tensorboard --logdir=.

In [165]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation="relu", input_shape=(30, 63)))
model.add(LSTM(128, return_sequences=True, activation="relu"))
model.add(LSTM(64, return_sequences=False, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(actions.shape[0], activation="softmax"))

In [125]:
res = [.7, .3]
actions[np.argmax(res)]

'Yes'

In [166]:
X_train.shape
y_train.shape

(608, 16)

In [167]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [168]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - categorical_accuracy: 0.0724 - loss: 2.7456
Epoch 2/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 0.2854 - loss: 2.5067
Epoch 3/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - categorical_accuracy: 0.3194 - loss: 2.2528
Epoch 4/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - categorical_accuracy: 0.3800 - loss: 1.9403
Epoch 5/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 0.5279 - loss: 1.6985
Epoch 6/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - categorical_accuracy: 0.5793 - loss: 1.1748
Epoch 7/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - categorical_accuracy: 0.5215 - loss: 1.2170
Epoch 8/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - categorical_accuracy: 0.6805 - loss: 0.8807
Epoch 9/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - categorical_accuracy: 0.7287 - loss: 1.0441
Epoch 10/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - categorical_accuracy: 0.8055 - loss: 0.5904
Epoch 11/

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - categorical_accuracy: 0.9840 - loss: 0.0470
Epoch 63/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - categorical_accuracy: 0.9913 - loss: 0.0214
Epoch 64/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 1.0000 - loss: 0.0097
Epoch 65/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 1.0000 - loss: 0.0046
Epoch 66/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - categorical_accuracy: 1.0000 - loss: 0.0044
Epoch 67/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - categorical_accuracy: 1.0000 - loss: 0.0030
Epoch 68/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - categorical_accuracy: 1.0000 - loss: 0.0026
Epoch 69/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 1.0000 - loss: 0.0017
Epoch 70/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 1.0000 - loss: 0.0016
Epoch 71/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - categorical_accuracy: 1.0000 - loss: 0.0014
Epoch 72/1000


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - categorical_accuracy: 0.9681 - loss: 0.1042
Epoch 123/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 0.9769 - loss: 0.0926
Epoch 124/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 0.9926 - loss: 0.0424
Epoch 125/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 0.9911 - loss: 0.0310
Epoch 126/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 0.9692 - loss: 0.0767
Epoch 127/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 0.9614 - loss: 0.1046
Epoch 128/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 0.9884 - loss: 0.0511
Epoch 129/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 0.9919 - loss: 0.0349
Epoch 130/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - categorical_accuracy: 0.9970 - loss: 0.0203
Epoch 131/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 0.9973 - loss: 0.0115
Epoch

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - categorical_accuracy: 1.0000 - loss: 2.0355e-04
Epoch 183/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - categorical_accuracy: 1.0000 - loss: 2.0965e-04
Epoch 184/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - categorical_accuracy: 1.0000 - loss: 2.9815e-04
Epoch 185/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - categorical_accuracy: 1.0000 - loss: 1.9856e-04
Epoch 186/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - categorical_accuracy: 1.0000 - loss: 2.3178e-04
Epoch 187/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - categorical_accuracy: 1.0000 - loss: 2.0488e-04
Epoch 188/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - categorical_accuracy: 1.0000 - loss: 2.0229e-04
Epoch 189/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 1.0000 - loss: 1.4569e-04
Epoch 190/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - categorical_accuracy: 1.0000 - loss: 1.5716e-04
Epoch 191/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_a

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - categorical_accuracy: 1.0000 - loss: 3.2957e-05
Epoch 242/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - categorical_accuracy: 1.0000 - loss: 2.4854e-05
Epoch 243/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - categorical_accuracy: 1.0000 - loss: 2.5604e-05
Epoch 244/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - categorical_accuracy: 1.0000 - loss: 2.7104e-05
Epoch 245/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 1.0000 - loss: 4.2885e-05
Epoch 246/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 1.0000 - loss: 3.7882e-05
Epoch 247/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - categorical_accuracy: 1.0000 - loss: 3.7247e-05
Epoch 248/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - categorical_accuracy: 1.0000 - loss: 2.2608e-05
Epoch 249/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - categorical_accuracy: 1.0000 - loss: 2.1772e-05
Epoch 250/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - categorical_

...

19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step - categorical_accuracy: 1.0000 - loss: 1.3280e-04
Epoch 717/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 107ms/step - categorical_accuracy: 1.0000 - loss: 1.6521e-04
Epoch 718/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - categorical_accuracy: 1.0000 - loss: 1.4611e-04
Epoch 719/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - categorical_accuracy: 1.0000 - loss: 1.5143e-04
Epoch 720/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - categorical_accuracy: 1.0000 - loss: 1.6728e-04
Epoch 721/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - categorical_accuracy: 0.9948 - loss: 0.0093
Epoch 722/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - categorical_accuracy: 0.8025 - loss: 0.8112
Epoch 723/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - categorical_accuracy: 0.8041 - loss: 0.5375
Epoch 724/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - categorical_accuracy: 0.9419 - loss: 0.2657
Epoch 725/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - categorical_accuracy: 0.974

19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - categorical_accuracy: 1.0000 - loss: 1.6260e-04
Epoch 776/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - categorical_accuracy: 1.0000 - loss: 1.2521e-04
Epoch 777/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - categorical_accuracy: 1.0000 - loss: 1.4399e-04
Epoch 778/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - categorical_accuracy: 1.0000 - loss: 1.7618e-04
Epoch 779/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - categorical_accuracy: 1.0000 - loss: 1.3758e-04
Epoch 780/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 1.0000 - loss: 1.3062e-04
Epoch 781/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 1.0000 - loss: 1.2485e-04
Epoch 782/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 1.0000 - loss: 1.3574e-04
Epoch 783/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - categorical_accuracy: 1.0000 - loss: 1.2123e-04
Epoch 784/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_a

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 1.0000 - loss: 3.2685e-05
Epoch 834/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - categorical_accuracy: 1.0000 - loss: 2.9197e-05
Epoch 835/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - categorical_accuracy: 1.0000 - loss: 3.2731e-05
Epoch 836/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - categorical_accuracy: 1.0000 - loss: 3.1245e-05
Epoch 837/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - categorical_accuracy: 1.0000 - loss: 2.7418e-05
Epoch 838/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - categorical_accuracy: 1.0000 - loss: 2.7583e-05
Epoch 839/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - categorical_accuracy: 1.0000 - loss: 3.1415e-05
Epoch 840/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - categorical_accuracy: 1.0000 - loss: 2.4692e-05
Epoch 841/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 1.0000 - loss: 2.4688e-05
Epoch 842/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_a

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - categorical_accuracy: 1.0000 - loss: 1.4600e-05
Epoch 892/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - categorical_accuracy: 1.0000 - loss: 1.3286e-05
Epoch 893/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - categorical_accuracy: 1.0000 - loss: 1.3159e-05
Epoch 894/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - categorical_accuracy: 1.0000 - loss: 1.0126e-05
Epoch 895/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 1.0000 - loss: 1.0917e-05
Epoch 896/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - categorical_accuracy: 1.0000 - loss: 1.4705e-05
Epoch 897/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 1.0000 - loss: 1.3002e-05
Epoch 898/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - categorical_accuracy: 1.0000 - loss: 1.1855e-05
Epoch 899/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - categorical_accuracy: 1.0000 - loss: 1.3451e-05
Epoch 900/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - categorical_a

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 1.0000 - loss: 6.1893e-06
Epoch 951/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 1.0000 - loss: 6.2996e-06
Epoch 952/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 1.0000 - loss: 5.7828e-06
Epoch 953/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - categorical_accuracy: 1.0000 - loss: 6.0381e-06
Epoch 954/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - categorical_accuracy: 1.0000 - loss: 6.3701e-06
Epoch 955/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - categorical_accuracy: 1.0000 - loss: 5.9410e-06
Epoch 956/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - categorical_accuracy: 1.0000 - loss: 5.9705e-06
Epoch 957/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - categorical_accuracy: 1.0000 - loss: 5.0342e-06
Epoch 958/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - categorical_accuracy: 1.0000 - loss: 5.8226e-06
Epoch 959/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - categorical_a

In [ ]:
# model.load_weights("sign_language.keras")

# 9. Make Predictions:

In [171]:
model.summary(line_length=100)

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                               ┃ Output Shape                    ┃           Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ lstm_15 (LSTM)                             │ (None, 30, 64)                  │            32,768 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ lstm_16 (LSTM)                             │ (None, 30, 128)                 │            98,816 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ lstm_17 (LSTM)                             │ (None, 64)                      │            49,408 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense_15 (Dense)                           │ (None, 64)                      │             4,160 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense_16 (Dense)                           │ (None, 32)                      │             2,080 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense_17 (Dense)                           │ (None, 16)                      │               528 │
└────────────────────────────────────────────┴─────────────────────────────────┴───────────────────┘

 Total params: 563,282 (2.15 MB)

 Trainable params: 187,760 (733.44 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 375,522 (1.43 MB)

In [172]:
results = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 724ms/step


In [173]:
actions[np.argmax(results[0])]

'Nothing'

In [174]:
actions[np.argmax(y_test[0])]

'Nothing'

# 10. Save Weights:

In [175]:
model.save("sign_language__2.keras")

# 11. Evaluation using Confusion Matrix and Accuracy:

In [176]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [177]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


In [178]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [179]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[30,  0],
        [ 0,  2]],

       [[30,  0],
        [ 0,  2]],

       [[27,  0],
        [ 0,  5]],

       [[30,  0],
        [ 0,  2]],

       [[27,  0],
        [ 0,  5]],

       [[30,  0],
        [ 0,  2]],

       [[31,  0],
        [ 0,  1]],

       [[31,  0],
        [ 0,  1]],

       [[29,  0],
        [ 0,  3]],

       [[30,  0],
        [ 0,  2]],

       [[29,  0],
        [ 0,  3]],

       [[29,  0],
        [ 0,  3]],

       [[31,  0],
        [ 0,  1]]], dtype=int64)

In [180]:
accuracy_score(ytrue, yhat)

1.0

In [196]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(results, actions, input_frame, colors):
    output_frame = input_frame.copy()
    output_frame = cv2.resize(output_frame, (1280, 960))
    
    for num, prob in enumerate(results):
        cv2.rectangle(output_frame, (0, num*40), (int(prob*100), 40+num*40), colors[num%3], -1)
        cv2.putText(output_frame, actions[num], (0, 32+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

# 12. Test Real Time:

In [ ]:
# from tensorflow.keras.models import load_model

sequence = []
sentence = []
predictions = []
threshold = 0.5

# model = load_model("sign_language__2.keras")

stream_url = "http://192.168.169.196:81/stream"
cap = cv2.VideoCapture(stream_url)

while True:
    _, frame = cap.read()

    frame, results = mediapipe_detections(frame, hands)
    draw_landmarks(frame, results, corner_color)

    right_hand = extract_keypoints(results)
    sequence.append(right_hand)
    sequence = sequence[-30:]
    
    if len(sequence) == 30:
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        predictions.append(np.argmax(res))
    
        if np.unique(predictions[-10:])[0] == np.argmax(res):
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
            
    frame = prob_viz(res, actions, frame, colors)
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━

In [208]:
cap.release()
cv2.destroyAllWindows()